# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmcons').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.ml.clustering import KMeans

In [0]:
df = sqlContext.sql("SELECT * FROM hack_data_csv")
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
summary|Session_Connection_Time| Bytes Transferred| Kali_Trace_Used|Servers_Corrupted| Pages_Corrupted| Location| WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
 count| 334| 334| 334| 334| 334| 334| 334|
 mean| 30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413| null|57.342395209580864|
 stddev| 14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697| 3.06352633036022| null| 13.41106336843464|
 min| 1.0| 10.0| 0| 1.0| 6.0|Afghanistan| 40.0|
 max| 60.0| 1330.5| 1| 10.0| 15.0| Zimbabwe| 75.0|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+

In [0]:
df.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)

In [0]:
df.head(1)

Out[5]: [Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [0]:
df.groupBy('Location').count().show()

+--------------------+-----+
 Location|count|
+--------------------+-----+
 Anguilla| 1|
 Paraguay| 2|
 Macao| 2|
Heard Island and ...| 2|
 Yemen| 1|
 Tokelau| 2|
 Sweden| 3|
French Southern T...| 3|
 Kiribati| 1|
 Guyana| 2|
 Philippines| 3|
 Malaysia| 2|
 Singapore| 1|
United States Vir...| 6|
 Turkey| 1|
 Western Sahara| 2|
 Malawi| 2|
 Iraq| 3|
Northern Mariana ...| 3|
 Germany| 1|
+--------------------+-----+
only showing top 20 rows

In [0]:
df.columns

Out[6]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
dfb = df.select('Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed')
dfb.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)

In [0]:
dfb.head(1)

Out[10]: [Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, WPM_Typing_Speed=72.37)]

In [0]:
ass = VectorAssembler(inputCols=dfb.columns,
                     outputCol='features')

In [0]:
fdf=ass.transform(dfb)
fdf.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures',)
scaler_model=scaler.fit(fdf)
ffdf = scaler_model.transform(fdf)

In [0]:
ffdf.head(1)

Out[15]: [Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), scaledFeatures=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]))]

In [0]:
kma = KMeans(featuresCol='scaledFeatures',k=3)
modela = kma.fit(ffdf)
modela.transform(ffdf).groupBy('prediction').count().show()

In [0]:
modela.transform(ffdf).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

In [0]:
kmb = KMeans(featuresCol='scaledFeatures',k=2)
modelb = kmb.fit(ffdf)
modelb.transform(ffdf).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+